In [ ]:
import pandas as pd
import numpy as np
from pybnesian import hc, CLGNetworkType, SemiparametricBNType, SemiparametricBN
#from drawdata import draw_scatter
import matplotlib.pyplot as plt
import math

from pymoo.core.problem import ElementwiseProblem

from pymoo.optimize import minimize

from bayesace.utils import *
from bayesace.algorithms.bayesace_algorithm import BayesACE
from bayesace.algorithms.face import FACE

from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [ ]:
df = pd.read_csv("toy-3class.csv")
df["class"] = df["z"].astype('category')
df = df.drop("z", axis = 1 )

feature_columns = [i for i in df.columns if i != "class"]
df[feature_columns] = StandardScaler().fit_transform(df[feature_columns].values)

In [ ]:
df = pd.read_csv("six_gaussians.csv")
df["class"] = df["class"].astype(str).astype('category')

feature_columns = [i for i in df.columns if i != "class"]
df[feature_columns] = StandardScaler().fit_transform(df[feature_columns].values)
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
learned = hc(df, bn_type = CLGNetworkType(), operators = ["arcs"], score = "validated-lik")
learned.fit(df)

'''naive = SemiparametricBN(df.columns)
feature_columns = [i for i in df.columns if i != "class"]
for i in [i for i in df.columns if i != "class"] :
    naive.add_arc("class", i)
learned_kde = hc(df, start = naive, operators = ["arcs", "node_type"], score = "validated-lik")
learned_kde.fit(df)'''





In [ ]:
from bayesace.models.multi_bnaf import Arguments, MultiBnaf

args = Arguments()
args.device = "cpu"
args.save = False
args.load = False
args.batch_dim = 2000
args.epochs = 5000
mbnaf = MultiBnaf(args, df)



In [ ]:
plt.imshow(get_probability_plot(mbnaf))

In [ ]:
predict_class(df, mbnaf)

In [ ]:
for i in learned.nodes() :
    print(i)
    print(learned_kde.cpd(i))

In [ ]:
instance = df.iloc[[0]]
instance["class"] = pd.Categorical(["a"], categories=learned.cpd("class").variable_values())
log_likelihood(instance,learned)

In [ ]:
n_vertex = 2
x_og = df.iloc[[0]]
bayesace = BayesACE(bayesian_network=learned, features= df.columns[:-1], n_vertex=n_vertex, chunks = 10, penalty=10, pop_size=100, likelihood_threshold=0.01, accuracy_threshold= 0.05, generations=20, verbose=False)
res = bayesace.run(x_og)
res_b = res

In [ ]:
x_1 = x_og.drop("class",axis = 1)
df_vertex = res_b.path
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
plt.plot(df_vertex.x,df_vertex.y,color = "red")
#plt.show()
plt.savefig("bace.png")

In [ ]:
x_og = df.iloc[[0]]
face = FACE(bayesian_network=learned, features= df.columns[:-1], dataset=df.drop("class", axis = 1), graph_type = "epsilon", chunks = 10, penalty=1, distance_threshold=0.75, accuracy_threshold= 0.05,  likelihood_threshold=0.35, verbose = False)
res = face.run(x_og)

In [ ]:
x_1 = x_og.drop("class",axis = 1)
df_vertex = res.path
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
limit = 3
plt.plot(df_vertex.x,df_vertex.y,color = "red")
plt.xlim((-3,3))
plt.ylim((-3,3))
plt.show()
#plt.savefig("face.png")

In [ ]:
sum(abs(res.path-res_b.path))

In [ ]:
path_likelihood_length(res.path, learned) - path_likelihood_length(res_b.path, learned)